In [1]:
from IPython.display import display
from ml_tools import SimpleAggregator, Hour
import pandas as pd

In [4]:
single_dataset = pd.read_csv("data/ml_tools_single_table_dataset/fraud.csv")
display(single_dataset.sample(10))

,id,card_id,store_id,datetime,amount,currency,customer_present,provider,lat,lng,region,country,fraud
75650,75650,16804,2960,2019-07-23 21:14:53,89641,SBD,True,VISA 16 digit,40.85158,122.74754,Haicheng,CN,False
85683,85683,23219,7658,2019-04-13 17:35:23,19167,SVC,False,American Express,-27.43380,-65.61427,Aguilares,AR,False
47712,47712,15802,4694,2019-04-15 07:21:59,32217,LTL,False,VISA 19 digit,37.65181,120.33063,Longgang,CN,False
13271,13271,18309,10,2019-06-23 17:31:13,92510,INR,False,VISA 16 digit,45.39402,9.29109,San Giuliano Milanese,IT,True
731,731,28089,5179,2019-04-16 01:55:31,72089,JPY,True,Maestro,52.14000,5.58472,Barneveld,NL,False
30364,30364,29505,6585,2019-04-11 07:29:35,65000,BBD,True,Diners Club / Carte Blanche,-7.51611,109.05389,Wangon,ID,False
92758,92758,11848,9063,2019-06-23 10:39:57,62312,RON,False,JCB 16 digit,50.75000,2.25000,Saint-Omer,FR,False
16096,16096,15734,4082,2019-01-24 15:32:49,98424,SLL,True,Maestro,10.20898,123.75800,Naga,PH,True
16458,16458,31676,1734,2019-05-10 18:13:34,36953,UYU,False,Diners Club / Carte Blanche,24.41288,76.56719,Aklera,IN,False
10529,10529,15451,606,2019-03-16 07:28:08,56210,CRC,False,VISA 16 digit,42.24113,-88.31620,Crystal Lake,US,False
